In [8]:
'''

Описание: Программа делает 
- Перебор значений из двумерной таблицы с использованием масок 
- Вычисление галактической долготы и галактической широты объектов
- Перевод прямого восхождения и склонения объектов в галактическую систему координат
- Рассчет расстояния от центра галактики до скоплений из массива данных
- Рассчет собственного движения в галактической системе координат

Автор: Daria Zholnirova
Дата: май 2025

'''

'\n\nОписание: Программа делает \n- Перебор значений из двумерной таблицы с использованием масок \n- Вычисление галактической долготы и галактической широты объектов\n- Перевод прямого восхождения и склонения объектов в галактическую систему координат\n- Рассчет расстояния от центра галактики до скоплений из массива данных\n- Рассчет собственного движения mu_l в галактической системе координат\n\nАвтор: Daria Zholnirova\nДата: май 2025\n\n\nПример использования:\nПрограмма считывает файл table.fit\n\n\n'

In [9]:
from astropy.io import fits
from astropy.visualization import ImageNormalize, ZScaleInterval, PercentileInterval
import numpy as np
from astropy.coordinates import Angle
from astropy import units as u
import math

#Constants
D = math.radians(23.13)
A = math.radians(192.86)

In [2]:
# Перебор типа, радиальной скорости, и возраста из данного массива
with fits.open('table.fit') as hdul:
    data = hdul[1].data  
    
    # Создаем маски для строк
    mask_1 = data['Type'] == 'o'
    mask_2 = data['RV'] != 0
    mask_3 = data['logAge50'] != 0
    mask_4 = (data['logAge50'] >= 7.24) & (data['logAge50'] <= 7.65)
    
    # Применяем маску к данным
    comb_mask = mask_1 & mask_2 & mask_3 & mask_4
    filtered_data = data[comb_mask]
    print(f"Оставлено строк: {len(filtered_data)} из {len(data)}")
    
    # Создаем новую таблицу с отфильтрованными данными
    new_hdu = fits.TableHDU(filtered_data)
    
    # Сохраняем результат в новый файл
    new_hdu.writeto('filtered_file.fits', overwrite=True)

Оставлено строк: 501 из 7167


In [10]:
# Перевод прямого восхождения и склонения в галактическую систему координат

# Создаем пустой массив для результатов
b = np.zeros(len(data))
la_cos = np.zeros(len(data))
la_sin = np.zeros(len(data))
sin_la = np.zeros(len(data))
cos_la = np.zeros(len(data))
    
# Проходим по каждой строке таблицы
for i in range(len(data)):
    delta = data['DE_ICRS'][i]
    alfa = data['RA_ICRS'][i]
        
    # Вычисляем b
    b[i] = math.asin(math.sin(delta)*math.sin(D)+math.cos(delta)*math.cos(D)*math.cos(alfa-A))

    # Вычисляем la
    la_cos[i] = math.acos((math.cos(delta)*math.sin(alfa-A)/math.cos(b[i])))
    la_sin[i] = math.asin((math.sin(delta)*math.cos(D)-math.cos(delta)*math.sin(D)*math.cos(alfa-A))/math.cos(b[i]))

    cos_la[i] = math.cos(la_cos[i])
    sin_la[i] = math.sin(la_sin[i])

In [4]:
# Пересчет прямого восхождения и склонения в галактические координаты

from astropy.coordinates import SkyCoord, Galactic

ra = filtered_data['RA_ICRS']
dec = filtered_data['DE_ICRS']

# Преобразование координат
coords = SkyCoord(ra = filtered_data['RA_ICRS']*u.deg, dec = filtered_data['DE_ICRS']*u.deg, frame ='icrs')

# Конвертация в галактическую систему
galactic_coords = coords.galactic

ra_gal = coords.ra.deg   # l
dec_gal = coords.dec.deg # b

# print (ra_gal[2])


In [11]:
# Рассчет расстояния от центра галактики до скопления
R0 = 8.32 #кпк

r_sun = filtered_data['dist50'] * 0.001 #массив расстояний от солнца до объекта в кпк
# print (r_sun[5])

R = np.zeros(len(filtered_data))
l_rad = np.radians(ra_gal)
b_rad = np.radians(dec_gal)

R = np.sqrt(
    R0**2 + 
    r_sun**2 * np.cos(b_rad)**2 - 
    2 * R0 * r_sun * np.cos(l_rad) * np.cos(b_rad))

# Проверка результата
# print("Пример расчёта R для первых 5 объектов:")
# print(R[:20])

mask = (R > 6) & (R < 15)
filtered_R = R[mask]  # Отфильтрованные расстояния
filtered_data_1 = filtered_data[mask]  # Соответствующие строки данных

print(f"Всего объектов до фильтрации: {len(R)}")
print(f"Осталось после отбора (6 < R < 15 кпк): {len(filtered_R)}")

# Создаем новую таблицу с отфильтрованными данными
new_hdu = fits.TableHDU(filtered_data_1)
    
# Сохраняем результат в новый файл
new_hdu.writeto('filtered_file.fits', overwrite=True)

# Все R называются filtered_R

Всего объектов до фильтрации: 501
Осталось после отбора (6 < R < 15 кпк): 499


In [6]:
# Рассчет собственного движения mu_l в галактической системе координат

mu_l = np.zeros(len(filtered_R))

ra = filtered_data_1['RA_ICRS']
dec = filtered_data_1['DE_ICRS']

# print(mu_alfa[5])

coords_1 = SkyCoord(
    ra = filtered_data_1['RA_ICRS'] * u.deg,          # Прямое восхождение
    dec = filtered_data_1['DE_ICRS'] * u.deg,        # Склонение
    pm_ra_cosdec = filtered_data_1['pmRA'] * u.mas/u.yr,  # Собственное движение по RA (уже включая cosδ)
    # pm_ra_cosdec = data['pmra'] * np.cos(np.radians(data['dec'])) если косинус не включен
    pm_dec  = filtered_data_1['pmDE'] * u.mas/u.yr        # Собственное движение по DEC
)

# Преобразование в галактическую систему координат
galactic_coords = coords_1.transform_to(Galactic())

# Извлечение собственных движений
mu_l = galactic_coords.pm_l_cosb        # Собственное движение по галактической долготе (mas/yr)
mu_b = galactic_coords.pm_b        # Собственное движение по галактической широте (mas/yr)

# print(mu_l[:5])